<a href="https://colab.research.google.com/github/jhlee93/AFCompetition/blob/main/reproducibility/pytorch_randomness_control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

### 가장 먼저 seed를 고정합니다.  
자세한 내용은 Pytorch 공식 문서를 참고해주세요.  
https://pytorch.org/docs/stable/notes/randomness.html

In [ ]:
# Set Seed
seed = 333
deterministic = True
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
if deterministic:
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

Device: cuda


In [ ]:
batch_size = 32
learning_rate = 0.0001
epoch_num = 3

train_data = datasets.MNIST(root = './mnist_data/',
                            train=True,
                            download=True,
                            transform=transforms.ToTensor())
test_data = datasets.MNIST(root = './mnist_data/',
                            train=False,
                            download=True,
                            transform=transforms.ToTensor())

print('num of train : ', len(train_data))
print('num of test : ', len(test_data))

train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                           batch_size = batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                           batch_size = batch_size, shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw

num of train :  60000
num of test :  10000


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1, padding='same')
        self.conv2 = nn.Conv2d(32, 64, 3, 1, padding='same')
        self.dropout = nn.Dropout2d(0.25)
        # (입력 뉴런, 출력 뉴런)
        self.fc1 = nn.Linear(3136, 1000)    # 7 * 7 * 64 = 3136
        self.fc2 = nn.Linear(1000, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


### 다음 결과는 seed=333으로 고정된 학습 및 평가 결과 입니다.  

```
Train Step : 1000	Loss : 0.063364  
Train Step : 2000	Loss : 0.201627  
Train Step : 3000	Loss : 0.109464  
Train Step : 4000	Loss : 0.012779  
Train Step : 5000	Loss : 0.015030  
Test Accuracy : 98.50%  
```

### 아래 셀을 실행시켜 무작위성이 통제 되었는지 확인해 봅니다.

In [ ]:
model = CNN().to(device)


optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

model.train()
i = 1
for epoch in range(epoch_num):
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if i % 1000 == 0:
            print(f"Train Step : {i}\tLoss : {loss.item():3f}")
        i += 1

model.eval()
correct = 0
for data, target in test_loader:
    data = data.to(device)
    target = target.to(device)
    output = model(data)
    prediction = output.data.max(1)[1]
    correct += prediction.eq(target.data).sum()

acc = correct / len(test_loader.dataset) * 100
print(f"Test Accuracy : {acc:.2f}%")


Train Step : 1000	Loss : 0.063364
Train Step : 2000	Loss : 0.201627
Train Step : 3000	Loss : 0.109464
Train Step : 4000	Loss : 0.012779
Train Step : 5000	Loss : 0.015030
Test Accuracy : 98.50%


# [런타임 - 다시 시작 및 모두 실행] 을 실행하여 결과가 재현되는지 다시 한번 확인해봅니다.